# Week 04 | Homework

Sunday , June 26, 2022 <br>
Assignment: [here](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/04-deployment/homework.md)
---

In [ ]:
# !pip freeze | grep scikit-learn
# scikit-learn==1.0.2
# scikit-learn-intelex==2021.20210714.120553

In [6]:
!pip install scikit-learn==1.0.2

ERROR: Could not find a version that satisfies the requirement scikit-learn-intelex==2021.20210714.120553 (from versions: 2021.2.1, 2021.2.2, 2021.2.3, 2021.3.0, 2021.4.0, 2021.5.1, 2021.5.3, 2021.6.2, 2021.6.3)
ERROR: No matching distribution found for scikit-learn-intelex==2021.20210714.120553


In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

## Q1 : What's the mean predicted duration for this dataset?
**Answer:** 16.191691679979066

In [15]:
y_pred.mean()

16.191691679979066

In [11]:
len(y_pred)

990113

In [17]:
df['ride_id'] = f'2021/02_' + df.index.astype('str')

In [20]:
df['predictions'] = y_pred

In [21]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration,ride_id,predictions
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,None,B00021,10.666667,2021/02_1,14.539865
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,None,B00021,14.566667,2021/02_2,13.740422
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82,129,None,B00021,7.950000,2021/02_3,15.593339
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1,225,None,B00037,13.800000,2021/02_4,15.188118
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1,61,None,B00037,8.966667,2021/02_5,13.817206
...,...,...,...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,-1,31,None,B01717,13.533333,2021/02_1037687,12.433246
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,-1,169,None,B01717,11.466667,2021/02_1037688,12.910885
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28,171,None,B03285,25.383333,2021/02_1037689,15.834923
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16,252,None,B03285,18.050000,2021/02_1037690,16.783176


In [25]:
df_result = df[['ride_id','predictions']].copy()

In [26]:
df_result

,ride_id,predictions
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206
...,...,...
1037687,2021/02_1037687,12.433246
1037688,2021/02_1037688,12.910885
1037689,2021/02_1037689,15.834923
1037690,2021/02_1037690,16.783176


In [27]:
df_result.to_parquet(
    "/home/mlops_zoomcamp/Notebooks/week04/df_result.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

## Q2 : What's the size of the output file?
**Answer:** 19M

In [28]:
!ls -lh df_result.parquet
# -rw-rw-r-- 1 mlops_zoomcamp mlops_zoomcamp 19M Jun 26 04:59 df_result.parquet

-rw-rw-r-- 1 mlops_zoomcamp mlops_zoomcamp 19M Jun 26 04:59 df_result.parquet


## Q3 : Which command you need to execute for that?
**Answer:**  `jupyter nbconvert Homework.ipynb --to python`

## Q4 : What's the first hash for the Scikit-Learn dependency?
**Answer:**  

## Q5 : What's the mean predicted duration for March 2021?
**Answer:**  16.30

In [2]:
!python ~/Notebooks/week04/predict.py --year 2021 --month 03
#Mean predict value for the year 2021 and month 03 is 16.30

Mean predict value for the year 2021 and month 03 is 16.30


## Q6 : What's the mean predicted duration for March 2021 using docker buit image?
Deployed the container in GCP Cloud Run using FastAPI <br>
https://nyc-taxi.anandpiyush.com/

<div>
<img src="../images/api.png" width="800"/>
</div>

**Answer:**  9.97 <br>
Mean predict value for the year 2021 and month 04 is 9.97

## Bonus: upload the result to the cloud

Processed file : predicted_YYYY-MM_time-stamp.parquet <br>
https://storage.googleapis.com/nyc-taxi-processed/predicted_2021-04_20220627-051729.parquet
<div>
<img src="../images/gcp2.png" width="800"/>
</div>